<a href="https://colab.research.google.com/github/JoaoGranja/Semantic_Segmentation/blob/master/pascal_semantic_segmentation_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
is_colab = True


# **Colab Preparation** 
Before handling the project, we need to install tensorflow/keras and pip packages. I also share my google drive to simplify the connection with my google drive account.



In [ ]:
if is_colab:
    #Package Installation and share Google Drive
    !pip install --upgrade pip
    #!pip install --upgrade keras
    !pip install keras-resnet
    !pip install tensorflow==2.4.0
    !pip install tensorflow-gpu==2.4.0
    !pip install tensorflow_addons
    !pip install keras==2.4

    !pip install keras-segmentation

    from google.colab import drive
    drive.mount('/content/drive')

     |████████████████████████████████| 1.6 MB 21.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20486 sha256=4cc0a868c55008c4ac704fa10edebafebe1a8e2c304d16f5876513a6374201e4
  Stored in directory: /root/.cache/pip/wheels/bd/ef/06/5d65f696360436c3a423020c4b7fd8c558c09ef264a0e6c575
Successfully built keras-resnet
     |████████████████████████████████| 394.7 MB 16 kB/s 
     |████████████████████████████████| 3.8 MB 27.8 MB/s 
     |████████████████████████████████| 462 kB 35.7 MB/s 
     |████████████████████████████████| 2.9 MB 34.2 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.41.0
    Uninstalling grpcio-1.41.0:
      Successfully uninstalled grpcio-1.41.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
  

 # **Configuration and imports**

The first thing I do is to import all modules we need for this project. I also use some configuration parameters to provide more flexibility to the tranining process

In this project I will be making use of the Keras library for creating our model and training it. I will also use Matplotlib for visualizing our dataset to gain a better understanding of the images we are going to be handling.



In [ ]:
# Generic Imports
import time
import gc
import logging, os
import sys
import random
import warnings
import pickle
from math import ceil
from tqdm import tqdm
from itertools import chain

# data processing and visualization library
import numpy as np
import pandas as pd
#import seaborn as sns
from IPython.display import clear_output
import matplotlib.pyplot as plt
import cv2 

# tensorflow and keras for DL model
import tensorflow as tf
import tensorflow_datasets as tfds
#logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Model, save_model, load_model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras import backend as K
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.utils import to_categorical, plot_model
#from keras.utils import multi_gpu_model
from tensorflow.keras.preprocessing.image import Iterator, load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers.experimental import preprocessing

if is_colab:
    from google.colab import files
    sys.path.append('/content/drive/MyDrive/colab/Semantic_Segmentation')
    os.chdir('/content/drive/MyDrive/colab/Semantic_Segmentation')
    
#------------------------------  Set some configuration parameters -----------------------------------#
args = {}
args['seed'] = 42
args['data_augmentation'] = False
args['training'] = True
args['fine_tuning'] = True
args['evaluation_networks'] = False
args['debug'] = False

#training arguments
args['train_epochs'] = 10
args['fine_tune_epochs'] = 10
args['validation_split'] = 0.2

#model arguments
args['compare_networks'] = ['mobileNetV2_Unet', 'mobilenet_fcn_32', 'mobilenet_fcn_8', 'mobilenet_pspnet', 
                            'mobileNetV2_pspnet', 'mobilenet_segnet', 'mobileNetV2_segnet', 'Deeplabv3']
args['network'] = 'Deeplabv3'
args['models_dir'] = 'nn_models_checkpoints'

#optimizer arguments
args['optimizer'] = 'adam'
#args['weights'] = 'nn_models_checkpoints/best_{}.h5'.format(args['networks'][0])
args['learning_rate'] = 0.001
args['decay'] = 0.0001
args['loss'] = SparseCategoricalCrossentropy(from_logits=True)

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')

## Loading the dataset

In this project I will use the CIFAR100 dataset which is comprised of 60000 32x32 color images in 100 classes, with 600 images per class. There are 50000 training images and 10000 test images. More information is available on [CIFAR homepage](https://www.cs.toronto.edu/~kriz/cifar.html)


# **Data pre-processing and data augmentation**
The dataset output ranges from class 0 to 99. For training a model, it is easier to use a one hot encoding for the class element of each sample, in this way we transform any integer into a 100 element binary vector with a 1 for the index of the class value.

In order to improve the model generalization for new unseen images, I will also apply data augmentation process based on random transformations.

In [ ]:
from dataset.VOC2012 import create_tf_record_voc, parse_record

create_tf_record_voc()

train_output_path = './dataset/voc12_train.record'
val_output_path = './dataset/voc12_val.record'
dataset = tf.data.Dataset.from_tensor_slices(train_output_path)
dataset = dataset.flat_map(tf.data.TFRecordDataset)

dataset = dataset.shuffle(buffer_size=1000)

dataset = dataset.map(parse_record)
#dataset = dataset.map(
#    lambda image, label: preprocess_image(image, label, is_training))
dataset = dataset.prefetch(10)

# We call repeat after shuffling, rather than before, to prevent separate
# epochs from blending together.
dataset = dataset.repeat(2)
dataset = dataset.batch(10)

iterator = dataset.make_one_shot_iterator()
images, labels = iterator.get_next()

plt.figure(figsize=(15, 15))

plt.subplot(1, 1, 1)
plt.title("Image")
plt.imshow(tf.keras.preprocessing.image.array_to_img(images))
plt.axis('off')

plt.subplot(1, 1, 2)
plt.title("labels")
plt.imshow(tf.keras.preprocessing.image.array_to_img(labels))
plt.axis('off')
plt.show()




ValueError: ignored

In [ ]:
def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  input_mask -= 1
  return input_image, input_mask

def load_image(datapoint, input_shape=128):
  input_image = tf.image.resize(datapoint['image'], (input_shape, input_shape))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (input_shape, input_shape))

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

class Augment(tf.keras.layers.Layer):
  def __init__(self, seed=42):
    super().__init__()
    # both use the same seed, so they'll make the same randomn changes.
    self.augment_inputs = preprocessing.RandomFlip(mode="horizontal", seed=seed)
    self.augment_labels = preprocessing.RandomFlip(mode="horizontal", seed=seed)

  def normalize(self, image, label):
    image = tf.cast(image, tf.float32) / 255.0
    #label -= 1
    return image, label

  def call(self, inputs, labels):
    inputs = self.augment_inputs(inputs)
    labels = self.augment_labels(labels)
    inputs, labels = self.normalize(inputs, labels)

    return inputs, labels

def crop_or_pad_image_and_label(image, label):
    """Crops and/or pads an image to a target width and height.
    Resizes an image to a target width and height by rondomly
    cropping the image or padding it evenly with zeros.
    Args:
      image: 3-D Tensor of shape `[height, width, channels]`.
      label: 3-D Tensor of shape `[height, width, 1]`.
    Returns:
      Cropped and/or padded image.
      If `images` was 3-D, a 3-D float Tensor of shape
      `[new_height, new_width, channels]`.
    """
    image_shape = 256
    label = label - 255  
    image_height = tf.shape(image)[0]
    image_width = tf.shape(image)[1]

    target_height = image_height + tf.maximum(image_shape - image_height, 0)
    target_width = image_width + tf.maximum(image_shape - image_width, 0)

    # Pad image/label
    image = tf.image.pad_to_bounding_box(image, 0, 0, target_height, target_width)
    label = tf.image.pad_to_bounding_box(label, 0, 0, target_height, target_width)
    
    # Crop image/label
    image_and_label = tf.concat([image, label], axis=2)
    image_and_label_crop = tf.image.random_crop(image_and_label, [image_shape, image_shape, 4])
    
    image_crop = image_and_label_crop[:, :, :3]
    label_crop = image_and_label_crop[:, :, 3:]
    label_crop += 255

    label_crop = tf.cast(label_crop, tf.int32)

    image_crop.set_shape([image_shape, image_shape, 3])
    label_crop.set_shape([image_shape, image_shape, 1])

    return image_crop, label_crop

In [ ]:
from dataset.create_pascal_tf_record import create_pascal_tf_record, parse_record

train_output_path = './dataset/voc_train.record'
val_output_path = './dataset/voc_val.record'
if not os.path.exists(train_output_path):
  create_pascal_tf_record()

use_pascal_dataset = True
if use_pascal_dataset:
  #------------------------------  Load dataset using TFRecordDataset -----------------------------------#
  train_dataset = tf.data.TFRecordDataset(train_output_path)
  train_images = train_dataset.map(parse_record).map(crop_or_pad_image_and_label)

  val_dataset = tf.data.TFRecordDataset(val_output_path)
  val_images = val_dataset.map(parse_record).map(crop_or_pad_image_and_label)
  BATCH_SIZE = 32
  BUFFER_SIZE = 100
  INPUT_SHAPE = [256,256,3]
  OUTPUT_CLASSES = 21
  STEPS_PER_EPOCH = 1464 // BATCH_SIZE
  VAL_SUBSPLITS = 5
  VALIDATION_STEPS = 1449 //BATCH_SIZE//VAL_SUBSPLITS

else:
  #------------------------------  Load dataset using tf.dataset API -----------------------------------#
  dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)
  train_images = dataset['train'].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
  val_images = dataset['test'].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)


  BATCH_SIZE = 64
  BUFFER_SIZE = 1000
  INPUT_SHAPE = [128,128,3]
  OUTPUT_CLASSES = 3
  STEPS_PER_EPOCH = info.splits['train'].num_examples // BATCH_SIZE
  VAL_SUBSPLITS = 5
  VALIDATION_STEPS = info.splits['test'].num_examples //BATCH_SIZE//VAL_SUBSPLITS


# Create Train and validation Batches
train_batches = (
    train_images
    .cache()                                 # The first time the dataset is iterated over, its elements will be cached either in the specified file or in memory. Subsequent iterations will use the cached data
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)                       # Combines consecutive elements of this dataset into batches.
    .repeat()                                # Repeats this dataset so each original value is seen count times.
    .map(Augment())
    .prefetch(buffer_size=tf.data.AUTOTUNE)) # This allows later elements to be prepared while the current element is being processed.

val_batches = val_images.batch(BATCH_SIZE)

In [ ]:
#------------------------------ Plot random images and respective true mask -----------------------------------#
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

for images, masks in train_batches.take(2):
  sample_image, sample_mask = images[0], masks[0]
  display([sample_image, sample_mask])


In [ ]:

def get_uniques(t):
    t1d = tf.reshape(t, shape=(-1,))
    # or tf.unique, if you don't need counts
    uniques, idx, counts = tf.unique_with_counts(t1d) 
    return uniques, idx, counts 




Analyzing the images it is clear that the images resolution is small, actually 32x32 has few pixels and therefore can be a challenge for a model to classify correctly the object. Furthermore all image has the same size so it is not required to resize the input images.

# **Optimizer**

Before training it is necessary to choose an optimizer which will be responsible to adjust model parameters in order to reduce the loss funcion

In [ ]:
#------------------------------ Define an optimizer -----------------------------------#
if 'optimizer' in args:
    if args['optimizer'] == 'rmsprop':
        optimizer = RMSprop(learning_rate=args['learning_rate'], decay=float(args['decay']))
    elif args['optimizer'] == 'adam':
        optimizer = Adam(learning_rate=args['learning_rate'], decay=float(args['decay']))
    elif args['optimizer'] == 'amsgrad':
        optimizer = Adam(learning_rate=args['learning_rate'], decay=float(args['decay']), amsgrad=True)
    elif args['optimizer'] == 'sgd':
        optimizer = SGD(learning_rate=args['learning_rate'], momentum=0.9, nesterov=True, decay=float(args['decay']))
else:
    optimizer = RMSprop(learning_rate=args['learning_rate'])

# **Model**

In this project, I will use simple and light CNN models. One model follows the LeNet architecture which is built from scratch and two models are the pre-trained VGG16 and VGG19 models.

In [ ]:
from models.model_factory import make_model

if args['training']:
  #------------------------------ Make the model -----------------------------------#
  model = make_model(args['network'], INPUT_SHAPE, OUTPUT_CLASSES)

  if 'weights' not in args:
      print('No weights passed, training from scratch')
  else:
      print('Loading weights from {}'.format(args['weights']))
      model.load_weights(args['weights'], by_name=True)

  #------------------------------ Compile the model -----------------------------------#
  model.compile(loss=args['loss'], optimizer=optimizer, metrics=['accuracy']) 

if args['debug']:
  model.summary()

In [ ]:
if args['debug']:
  plot_model(model, show_shapes=True)

# **Training**

This step called trainig comprises on fitting the model parameters to classify correcty the images. For that, the optimizer uses a loss function to quantify the discrepance between true and predicted labels Based on that it adjusts the model parameters to decrease the loss. Model checkpoints, EarlyStopping and Learning Rate reduce are used as callbacks to improce training efficiency.

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image[0], mask[0], create_mask(pred_mask)])
  else:
    display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])


class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    show_predictions()
    print ('\nSample Prediction after epoch {}\n'.format(epoch+1))


def add_sample_weights(image, label):
  # The weights for each class, with the constraint that:
  #     sum(class_weights) == 1.0
  class_weights = tf.constant([2.0, 2.0, 1.0])
  class_weights = class_weights/tf.reduce_sum(class_weights)

  # Create an image of `sample_weights` by using the label at each pixel as an 
  # index into the `class weights` .
  sample_weights = tf.gather(class_weights, indices=tf.cast(label, tf.int32))

  return image, label, sample_weights

In [ ]:
gc.collect
#------------------------------ Model check points -----------------------------------#
best_model_file = '{}/best_{}.h5'.format(args['models_dir'], model.name)
last_model_file = '{}/last_{}.h5'.format(args['models_dir'], model.name)
model_file = '{}/model_{}.h5'.format(args['models_dir'], model.name)

#------------------------------ Callbacks -----------------------------------#

#ModelCheckpoint(filepath=last_model_file, monitor='val_loss', verbose=1, mode='min',
#            save_freq='epoch', save_best_only=False, save_weights_only=False)

# Callback to reduce the learning rate once the plateau has been reached:
reduceLr = ReduceLROnPlateau(
    monitor='val_loss',
    min_delta=0.05,
    factor=1/3,
    patience=3,
    mode='auto',
    verbose=1,
    cooldown=0,
    min_lr=1e-8
),
# Callback to stop the training once no more improvements are recorded:
earlyStopping = EarlyStopping(
    min_delta=0.001,
    verbose=1,
    patience=10,
    mode='auto',
    restore_best_weights=True
),
# Callback to log the graph, losses and metrics into TensorBoard:
tensorBoard = TensorBoard(log_dir="logs/{}".format(model.name)
),
# Callback to save the best model specifying the epoch and val-loss in the filename:
model_checkPoint = ModelCheckpoint(filepath=best_model_file, 
    monitor='val_loss',
    verbose=1,
    mode='min',
    save_freq='epoch',
    save_best_only=True,
    save_weights_only=False)


callbacks = [reduceLr, earlyStopping, model_checkPoint, DisplayCallback()] #tensorBoard, 

if args['training']:
    print("Training model {}".format(model.name))
    #------------------------------ Model Fit -----------------------------------#
    history = model.fit(
                        train_batches,
                        steps_per_epoch=STEPS_PER_EPOCH,
                        epochs=args['train_epochs'],
                        validation_steps=VALIDATION_STEPS,
                        validation_data=val_batches,
                        callbacks=callbacks)

    #------------------------------ Save the last model weights -----------------------------------#
    model.save(model_file)
    print("Saved model to disk") 
else:
  model = load_model(model_file)

# **Fine tuning**

In the feature extraction experiment, you were only training a few layers on top of an base model. The weights of the pre-trained network were not updated during training. 

One way to increase performance even further is to train (or "fine-tune") the weights of the top layers of the pre-trained model alongside the training of the classifier you added. The training process will force the weights to be tuned from generic feature maps to features associated specifically with the dataset.

In [ ]:
if args['fine_tuning']:
  #base_model = model.get_layer('encoder')
  #base_model.trainable = True

  # Let's take a look to see how many layers are in the base model
  #print("Number of layers in the base model: ", len(base_model.layers))

  # Fine-tune from this layer onwards
  #fine_tune_at = 100

  # Freeze all the layers before the `fine_tune_at` layer
  #for layer in base_model.layers[:fine_tune_at]:
   # layer.trainable =  False

  #------------------------------ Compile the model -----------------------------------#
  model.compile(loss=args['loss'], optimizer=RMSprop(learning_rate=args['learning_rate']/10), metrics=['accuracy']) 
  model.summary()

  #------------------------------ Model Fit -----------------------------------#
  total_epochs =  args['train_epochs'] + args['fine_tune_epochs']
  fine_tune_history = model.fit(
                        train_batches.map(add_sample_weights),
                        steps_per_epoch=STEPS_PER_EPOCH,
                        epochs=total_epochs,
                        initial_epoch = args['train_epochs'],
                        validation_steps=VALIDATION_STEPS,
                        validation_data=test_batches,
                        callbacks=callbacks)

  #------------------------------ Save the last model weights -----------------------------------#
  model.save(model_file)
  print("Saved model to disk")    


# **Visualise Model Results**

In [ ]:
#------------------------------ Plot diagnostic learning curves -----------------------------------#
def summarize_diagnostics(history, model_name, fine_tune_history={}):
    # plot loss
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    if args['fine_tuning']:
      acc += fine_tune_history.history['accuracy']
      val_acc += fine_tune_history.history['val_accuracy']

      loss += fine_tune_history.history['loss']
      val_loss += fine_tune_history.history['val_loss']  

    plt.figure(figsize=(8, 8))
    plt.subplot(2,1,1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.ylim([0.1, 1])
    if args['fine_tuning']:
      plt.plot([args['train_epochs'],args['train_epochs']],
          plt.ylim(), label='Start Fine Tuning')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2,1,2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    #plt.ylim([0, 10])
    if args['fine_tuning']:
      plt.plot([args['train_epochs'],args['train_epochs']],
          plt.ylim(), label='Start Fine Tuning')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    
    # save plot to file
    plt.savefig("results_" + model_name  + '_plot.png')
    plt.show()
    plt.close()

if args['training']:  
    if args['fine_tuning']:
      summarize_diagnostics(history, args['network'], fine_tune_history)
    else:
      summarize_diagnostics(history, args['network'])



In [ ]:
show_predictions(test_batches, 3)

# **Evaluation**

Evaluate the model over the test dataset. We will use the last model weights and predict the class for some test images

In [ ]:
gc.collect
if args['evaluation_networks']:

  def show_image_prediction(loaded_model, n_images):
    #Try out the model on an image from the test data:
    #plt.figure(figsize=(30, 30))
    fig, axs = plt.subplots(1, n_images, figsize=(15, 15))
    fig.tight_layout(pad=1.0)

    # View the images
    for i in range(n_images):
        index = random.randint(0, len(x_test))
        image = x_test[index].squeeze()
        true_index = [i for i in range(nr_classes) if y_test[index][i] == 1 ][0]

        prediction_scores = loaded_model.predict(np.expand_dims(image, axis=0))
        predicted_index = np.argmax(prediction_scores)

        #image = np.add(image*128,128).astype(int)
        axs[i].imshow(image)
        axs[i].set_title("True Label = {0}, \n Predicted label = {1}".format(true_index, predicted_index))
        axs[i].get_xaxis().set_visible(False)
        axs[i].get_yaxis().set_visible(False)
    
    plt.show()
    plt.close()

  #------------------------------ Predict on some test images -----------------------------------#
  i=1

  for network in args['compare_networks']:
      print("----------------------- model {} -----------------------".format(network))


      #plt.subplot(1, len(args['networks']), i)
      #Load last model parameters
      last_model_file = '{}/last_{}.h5'.format(args['models_dir'], network)
      best_model_file = '{}/best_{}.h5'.format(args['models_dir'], network)
      model_file = '{}/model_{}.h5'.format(args['models_dir'], network)

      #Load the model
      loaded_model = load_model(model_file)
      print("Loaded model {} from disk".format(model_file))
      loaded_model.compile(loss=args['loss'], optimizer=optimizer, metrics=['accuracy']) 

      #Try out the model on an image from the test data:
      show_image_prediction(loaded_model,8)

      #------------------------------ Evaluate model on testing dataset -----------------------------------#
      #_, acc = loaded_model.evaluate(x_train, y_train, verbose=0)
      #print("Training accuracy of model {0} = {1}".format(loaded_model.name, acc))
      _, acc = loaded_model.evaluate(x_test, y_test, verbose=0)
      print("Testing accuracy of model {0} = {1}".format(loaded_model.name, acc)) 

      #------------------------------ Plot all model results -----------------------------------#
      # save plot to file
      filename = "accuracy_" + network + '_plot.png'
      im = cv2.imread(filename)
      plt.imshow(im)
      plt.title("Accuray Results of model {}".format(network))
      i = i + 1

      plt.show()
  plt.close()

Comparing the results obtained from the three models *LeNet*, *VGG16* and *VGG19*, it is possible to conclude the model with the best result is *VGG16* followed by *VGG19*. Indeed *VGG16* has the highest accuracy evaluated on testing dataset. Due to reduced capacity, *LeNet* starts overfiting earlier, actually the fine-tuning process didn't improve model results.

**Conclusion**

In this project I built from scratch a simple LeNet model and used 2 pre-trained models, VGG16 and VGG19. I train all models and evaluate the results against the CIFAR10 testing dataset. Taking into account the accuracy, the best model for this dataset was VGG16. 

So in this project, I applied several workflow steps to train a deep learning model on the image classification problem. In sume the steps were: pre-processing the input data, apply data-augmentation, build a CNN model, use pre-trained models, train a model, fine-tuning the model and evaluate the model. 

I used transfer learning technique on VGG16 and VGG19 models which is usefull when our dataset is not so large like in this case. The models accuracy on testing dataset is around 81/83% which are good results. 

For future work, I could apply better data augmentation in order to avoid overfitting as well as regularization techniques like L1 or L2. 